# Ohio Alleycat Resource Data Analysis
---

## In this file we will analyze adoption data collected from Ohio Alleycat Resource. We will inspect and clean three separate dataframes, relating to animals, adoptions, and adopters. We will merge these three into one dataframe to be used for analysis in Tableau.
---

In [ ]:
#import the relevant python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Show all columns of out dataframe
pd.set_option("max_columns", None)

---
## First we will analyze the adopters data. 

In [ ]:
adopters = pd.read_csv('./data/adopters.csv')

In [ ]:
adopters.head()

In [ ]:
adopters.info()

---
### We can see that the OWNERCOUNTY value actually contains the state of the adress field, but the formatting of some values is different and there are various mispellings. For example, "OH" and "Ohio".

In [ ]:
adopters['OWNERCOUNTY'].value_counts()

### First, we will address the various forms of Ohio and replace them all with OH. 

In [ ]:
adopters['OWNERCOUNTY'].replace(['Ohio', 'Oh', 'oh', 'ohio', 'OHIO', 'Ohip', 'OH.US', 'Ohiop', 'OH - Ohio', 'OB', 'Oh,ohio', 'Ohi', 'Ohii'], 'OH', inplace = True)

In [ ]:
adopters['OWNERCOUNTY'].value_counts()

### That fixed the Ohio values. We will also fix the problems with Kentucky and Indiana, and replace other states with their abbreviations.

In [ ]:
adopters['OWNERCOUNTY'].replace(['Kentucky', 'Ky', 'ky', 'KENTUCKY', 'kentucky', 'Ky - Kentucky'], 'KY', inplace = True)

In [ ]:
adopters['OWNERCOUNTY'].replace(['Indiana', 'Indiana (IN)'], 'IN', inplace = True)

In [ ]:
adopters['OWNERCOUNTY'].replace(['Illinois', 'Alabama', 'Massachusetts', 'North Carolina'], ['IL', 'AL', 'MA', 'NC'], inplace = True)

In [ ]:
adopters['OWNERCOUNTY'].value_counts()

### There is still one value of "US". Let's see what state they should be.

In [ ]:
adopters.loc[adopters['OWNERCOUNTY'] == 'US']

### Bellevue is in Kentucky, so we will change this to KY.

In [ ]:
adopters['OWNERCOUNTY'].replace('US', 'KY', inplace = True)

---
### Next we will strip the address from the Latitude/Longitude field and split the Latitude and Longitude into separate fields.

In [ ]:
#Split the LATLONG column on the commas into a list with three values. 
#The latitude is the first element and the Longitude is the second element of the list
#The third element is the address which we discard.
adopters['LATITUDE'] = adopters['LATLONG'].str.split(',').str[0]

In [ ]:
adopters['LONGITUDE'] = adopters['LATLONG'].str.split(',').str[1]

In [ ]:
#We can see the Latitude and Longitude columns were added
adopters.head()

In [ ]:
#Check for NaN in Latitude
adopters['LATITUDE'].isnull().value_counts()

### We can see there are some Null values in the Latitude, so we will fill them with zeros.

In [ ]:
adopters['LATITUDE'] = adopters['LATITUDE'].fillna('0')

In [ ]:
adopters['LATITUDE'].isnull().value_counts()

### And also fill the Null values of Longitude with zeros.

In [ ]:
adopters['LONGITUDE'].isnull().value_counts()

In [ ]:
adopters['LONGITUDE'] = adopters['LONGITUDE'].fillna('0')

In [ ]:
adopters['LONGITUDE'].isnull().value_counts()

---
### Finally, we will rename some of the columns.

In [ ]:
col_map = {'OWNERADDRESS': 'ADDRESS', 'OWNERTOWN': 'CITY', 'OWNERCOUNTY': 'STATE', 'OWNERPOSTCODE': 'ZIPCODE'}
adopters.rename(columns = col_map, inplace=True)

### We will also drop the columns we don't need.

In [ ]:
adopters.drop(columns=['LATLONG'], axis = 1, inplace = True)

---
## Now we will analyze the Animals data.

In [ ]:
animals = pd.read_csv('./data/animals.csv')

In [ ]:
animals.head(10)

**Note: duplicate cats, such as Orry seen above, were returned from their orginal adoption and adopted again, so we will not remove the duplicate values.
This is Orry:**
<div>
<img src="./photos/orry.jpeg" width="200"/>
</div>

In [ ]:
animals.info()

---
### First we will address some of the coded values and change them to values that are easier to interpret. We will change the values of the 'Sex' column to male and female. 1 is male and 0 is female. 

In [ ]:
animals['SEX'].replace([0, 1], ['Female', 'Male'], inplace = True)

### We will also change the values of the 'Declawed' column. 0 = not declawed, 1 = declawed.

In [ ]:
animals['DECLAWED'].replace([0, 1], ['Not Declawed', 'Declawed'], inplace = True)

### Next we will fill in the data for the Combitest Result. A result of 2 means the cat tested positive for Feline Immunodefciency Virus. Otherwise the cat is negative.  We will also rename this column.

In [ ]:
animals['COMBITESTRESULT'].replace([[0,1],2], ['FIV-','FIV+'], inplace = True)

### And also we will fill in the data for the FLVRESULT field. A result of 2 means that cat tested positive for Feline Leukemia Virus. Otherwise the cat is negative. We will also rename this column.

In [ ]:
animals['FLVRESULT'].replace([[0,1],2], ['FELV-','FELV+'], inplace = True)

### To check this we will look for a cat that is FIV+ and FELV-, Otto Tunes. Here is a photo of this handsome gentleman:
<div>
<img src="./photos/otto.jpeg" width="200"/>
</div>

In [ ]:
animals.loc[animals['ANIMALNAME'] == 'Otto Toons']

In [ ]:
animals.rename(columns = {'COMBITESTRESULT': 'FIV_STATUS', 'FLVRESULT': 'FELV_STATUS'}, inplace = True)

### We can see that the replacement worked as intended.
---

### To get the Base Color id, we will merge a dataframe that contains the Base Color table.

In [ ]:
color = pd.read_csv('./data/coat_color.csv')

In [ ]:
color['BASECOLOUR'].value_counts()

### This is a mess! We will replace some of the values that seem to be identical, such as "White and Orange Tabby" and "Orange and White Tabby", to reduce the number of unique values.

In [ ]:
color['BASECOLOUR'].replace(['White and Orange Tabby', 
                             'White and Buff Tabby', 
                             'White and Brown Tabby', 
                             'Brown Tabby with White', 
                             'White and Gray Tabby', 
                             'White and Silver Tabby', 
                             'White and Smoke Tabby',
                             'White and Torbie',
                             'White and Gray',
                             'White and Black'], 
                            ['Orange Tabby and White', 
                             'Buff Tabby and White', 
                             'Brown Tabby and White', 
                             'Brown Tabby and White',
                             'Gray Tabby and White', 
                             'Silver Tabby and White', 
                             'Smoke Tabby and White',
                             'Torbie and White',
                             'Gray and White',
                             'Black and White'],
                           inplace = True)

In [ ]:
color['BASECOLOUR'].sort_values().value_counts()

### We need to rename the 'ID' column in  order to merge on the 'BASECOLOURID' in the animals dataframe.

In [ ]:
color.rename({'ID': 'BASECOLOURID'}, axis = 1, inplace = True)

### We can drop the columns we don't need.

In [ ]:
color.drop(columns=['BASECOLOURDESCRIPTION', 'ADOPTAPETCOLOUR', 'ISRETIRED'], axis = 1, inplace = True)

### Now we can merge the color and animals dataframes.

In [ ]:
animals = animals.merge(color, on = 'BASECOLOURID', how = 'left')

In [ ]:
animals.head()

### We can see the BASECOLOUR Column was added as intended.
---

### We will repeat these steps to get the shelter location name by merging with the shelter location table.

In [ ]:
shelter_location = pd.read_csv('./data/shelter_location.csv')

In [ ]:
shelter_location.rename({'ID': 'SHELTERLOCATION'}, axis = 1, inplace = True)

In [ ]:
shelter_location.drop(columns=['LOCATIONDESCRIPTION', 'UNITS', 'ISRETIRED', 'SITEID'], axis = 1, inplace = True)

In [ ]:
animals = animals.merge(shelter_location, on = 'SHELTERLOCATION', how = 'left')

In [ ]:
animals.head()

---
## Next we will bring in the adoptions data.

In [ ]:
adoptions = pd.read_csv('./data/adoptions.csv')

In [ ]:
adoptions.head()

### The movement date is the date of the adoption, so we will rename that column to be more explanatory.

In [ ]:
adoptions.rename({'MOVEMENTDATE': 'ADOPTIONDATE'}, axis = 1, inplace = True) 

In [ ]:
adoptions.head()

---
# Now that we have the adopters animal, and adoption dataframes in good shape, we will merge them all into one dataframe called oar_data. 

# The adopters, animal, and adoption dataframes have three columns in common that we will use to merge: Adoption Number, Owner ID, and Animal ID.

In [ ]:
oar_data = adoptions.merge(adopters, how='inner', on = ['ADOPTIONNUMBER', 'ANIMALID', 'OWNERID']).merge(animals, how='inner', on = ['ADOPTIONNUMBER', 'ANIMALID', 'OWNERID'])

In [ ]:
oar_data.info()

### To check this, I am going to look up an adoption that I processed recently for a cat named Imogene. 

In [ ]:
oar_data.loc[oar_data['ANIMALNAME'] == 'Imogene']

### Since I handled this adoption personally, I can verify that all the data is correct.

**Here's Imogene with her new Dad:**
<div>
<img src="./photos/imogene.jpeg" width="200"/>
</div>

---
# Now we will analyze the data in the oar_data dataframe.

### First, we will calculate a new field that will calulate the Time to Adoption as the Adoption Date - the Date the cat was originally brought in to the shelter. 
### However, if the cat was returned, the Time to Adoption is the Adoption Date - the Most Recent Entry Date. We will need to use a function to calculate this.

### To easily identify the retuned cats, we will add a flag for those who were returned.

In [ ]:
oar_data['ADOPTIONDATE'] = oar_data['ADOPTIONDATE'].fillna('0')

In [ ]:
oar_data['DATEBROUGHTIN'] = oar_data['DATEBROUGHTIN'].fillna('0')

In [ ]:
oar_data['MOSTRECENTENTRYDATE'] = oar_data['MOSTRECENTENTRYDATE'].fillna('0')

In [ ]:
def was_returned(return_date):
    if return_date == 'None':
        return "Not returned"
    else:
        return "Returned"

In [ ]:
oar_data['RETURNED'] = oar_data['RETURNDATE'].apply(was_returned)

In [ ]:
oar_data.head()

In [ ]:
# Function to calculate the time to adoption based on Returned status
def time_to_adoption(return_status, adoption_date, entry_date, brought_in_date):
    if return_status == 'Not returned':
        return adoption_date - entry_date
    else:
        return adoption_date - brought_in_date

In [ ]:
#Using np.vectorize which is faster than df.apply
oar_data['TIME_TO_ADOPTION'] = np.vectorize(time_to_adoption)(oar_data['RETURNED'], pd.to_datetime(oar_data['ADOPTIONDATE']), pd.to_datetime(oar_data['MOSTRECENTENTRYDATE']), pd.to_datetime(oar_data['DATEBROUGHTIN']))

In [ ]:
oar_data['TIME_TO_ADOPTION'] = oar_data['TIME_TO_ADOPTION'].dt.days

In [ ]:
#set any negative values equal to 0
oar_data.loc[oar_data['TIME_TO_ADOPTION'] < 0, 'TIME_TO_ADOPTION'] = 0

In [ ]:
oar_data['TIME_TO_ADOPTION'].describe()

### Who is the cat who has a time to adoption of 5264 days?

In [ ]:
oar_data.loc[oar_data['TIME_TO_ADOPTION'] == 5264]

In [ ]:
oar_data.loc[oar_data['ANIMALNAME'] == 'Lewis']

### The date Lewis was brought into the shelter is listed as 1/1/01 which is surely incorrect. Someone must have entered this date as a placeholder because the original date was unknown.

### We will also check the results of our function by looking at a cat that was returned twice, named Danica Patrick.

In [ ]:
oar_data.loc[oar_data['ANIMALNAME'] == 'Danica Patrick']
# I don't have a photo of Danica but she is a great cat and was returned due to no fault of her own.

### We can see that our function doesn't work for the middle adoption, since the Most Recent Entry Date is overwritten in the database each time the cat is returned to the shelter. But the function works with the first and last adoptions, which is the best we can do here. 

### We will add new fields for the Month and Year of the adoption. 

In [ ]:
oar_data['ADOPTION_MONTH'] = pd.to_datetime(oar_data['ADOPTIONDATE']).dt.month

In [ ]:
oar_data['ADOPTION_YEAR'] = pd.to_datetime(oar_data['ADOPTIONDATE']).dt.year

In [ ]:
oar_data.head()

### And a field to calculate the age of the cat at the time of adoption.

In [ ]:
oar_data['AGE_AT_ADOPTION'] = (pd.to_datetime(oar_data['ADOPTIONDATE']) - pd.to_datetime(oar_data['DATEOFBIRTH'])).dt.days

In [ ]:
oar_data.head()

---
### Finally, we will save our complete dataset so that it can be used in Seaborn and Tableau.

In [ ]:
oar_data.to_csv('./data/oar_data.csv')

---
### Now we will look at some group operations. First, the adoptions by year.

In [ ]:
adoptions_by_year = oar_data.groupby('ADOPTION_YEAR').agg({'ADOPTIONNUMBER': 'count'})

In [ ]:
adoptions_by_year 

In [ ]:
adoptions_by_year_by_sex = oar_data.groupby(['ADOPTION_YEAR', 'SEX']).agg({'ADOPTIONNUMBER': 'count'})

### We can also look at the adoptions by month.

In [ ]:
adoptions_by_month = oar_data.groupby('ADOPTION_MONTH').agg({'ADOPTIONNUMBER': 'count'})

In [ ]:
adoptions_by_month

### Adotions by year for Male and Female Cats

In [ ]:
adoptions_by_year_by_sex

### Let's make a pivot table showing the adoptions by year and by month.

In [ ]:
pd.pivot_table(oar_data, index='ADOPTION_YEAR', values=['ADOPTIONNUMBER'], aggfunc='count', columns=['ADOPTION_MONTH'], fill_value = 0)

### We can see the change in 2014, when the current OAR shelter location opened.
---

### We will also load the data to Seaborn and make some graphs.

In [ ]:
#Count of Adoptions by year
sns.set_theme()
plt.figure(dpi=150)
sns.countplot(x ='ADOPTION_YEAR', data = oar_data)
plt.xticks(rotation = 45)
plt.title('Adoptions by Year')

In [ ]:
#Count of adoptions by year by sex
sns.set_theme()
plt.figure(dpi=150)
sns.countplot(x = 'ADOPTION_YEAR', data = oar_data, hue = 'SEX')
plt.xticks(rotation=45)
plt.title('Adoptions by Year by Sex')

In [ ]:
#Count of adoptions by returned/unreturned status
sns.set_theme()
plt.figure(dpi=150)
sns.countplot(x = 'ADOPTION_YEAR', data = oar_data, hue = 'RETURNED')
plt.xticks(rotation=45)
plt.title('Adoptions by Year by Return Status')

---
## Next we will look at the adoption numbers by some characteristics such as breed, coloring, and shelter location.
---

### Adoptions by breed

In [ ]:
adoptions_by_breed = oar_data.groupby('BREEDNAME').agg({'ADOPTIONNUMBER': 'count'})

In [ ]:
adoptions_by_breed.sort_values('ADOPTIONNUMBER', ascending = False)

In [ ]:
#Count of adoptions by breed
sns.set_theme()
plt.figure(dpi=150)
sns.countplot(x = 'BREEDNAME', data = oar_data)
plt.xticks(rotation=90)
plt.title('Adoptions by Breed')

### Adoptions by Coloring

In [ ]:
adoptions_by_color = oar_data.groupby(oar_data['BASECOLOUR']).agg({'ADOPTIONNUMBER': 'count'})

In [ ]:
adoptions_by_color.sort_values('ADOPTIONNUMBER', ascending = False)

In [ ]:
#Count of adoptions by coloring
sns.set_theme()
plt.figure(dpi=150)
sns.countplot(x = 'BASECOLOUR', data = oar_data)
plt.xticks(rotation=90)
plt.title('Adoptions by Coloring')

### Adoptions by Shelter Location

In [ ]:
adoptions_by_location = oar_data.groupby('LOCATIONNAME').agg({'ADOPTIONNUMBER': 'count'})

In [ ]:
adoptions_by_location.sort_values('ADOPTIONNUMBER', ascending = False)

In [ ]:
#Count of adoptions by shelter location
sns.set_theme()
plt.figure(dpi=150)
sns.countplot(x = 'LOCATIONNAME', data = oar_data)
plt.xticks(rotation=90)
plt.title('Adoptions by Shelter Location')

### Next we will look at the adoptions by year for cats who have Feline Leukemia Virus (FELV) and Feline Immunodeficiency Virus (FIV). 

### Cats with FELV must be isolated in the shelter and can only live with other cats who have FELV or other animals, since FELV is a serious illness that can be spread through casual contact such as sharing food and water bowls. 

### Cats with FIV can live with cats who do not have FIV since it is considered a less serious illness, generally does not affect the cat's life span, and is not spread through casual contact. 

In [ ]:
pd.pivot_table(oar_data, index=['FELV_STATUS','FIV_STATUS'], values=['ADOPTIONNUMBER'], aggfunc='count', columns=['ADOPTION_YEAR'], fill_value = 0)

### Finally we will look at how the age of the cat interacts with the time spent in the shelter. 

In [ ]:
#Age at adoption vs. Days spent in shelter
sns.set_theme()
plt.figure(dpi=200)
plt.title('Age in Days at Adoption vs. Days Spent in Shelter')
sns.scatterplot(data=oar_data, x='AGE_AT_ADOPTION', y='TIME_TO_ADOPTION')

### Even though we know some of the high values may be due to data entry errors, this still gives us a good impression of how the time to adoption generally increases with age. 

### Next we will take it over to Tableau!